# Pancreatitis scRNA-seq Data - Reananalysis

The data analysed in this notebook is from Poggetto E; Ho I et al Science 2025 (PMID:34529467). Data can be found on GEO under accession number GSM5494073. ScRNA-seq data are stored in the series GSE181276. The link below can be found by going to https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE181276 and with right click copy the "http" hyperlink in the Download column. 

The flags are required to download this file otherwise it downloads only a HTML page: <br>
-L --> redirects it to the actual file (*_genes.counts_*.txt.gz), without it would only download a HTML page. <br>
-O --> tells curl that it saves the file using the URL filename (without it would be in printed in the terminal) <br>
-J --> tells curl to use the name instead of the final url segment

In [1]:
#Create new direcotry for rawdata
!mkdir -p rawdata
# Download data using curl
!cd rawdata && curl -L -O -J "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE181276&format=file&file=GSE181276_genes.counts_for_GEO_uploading.txt.gz"
# Unzip the gene_counts file in the rawdata folder
!cd rawdata && gunzip "GSE181276_genes.counts_for_GEO_uploading.txt.gz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  105M  100  105M    0     0  23.7M      0  0:00:04  0:00:04 --:--:-- 25.0M


In [2]:
#Import of software packages
import scanpy as sc
import scvi

In [15]:
# Read file but transpose it as Anndata and Scanpy require cells (barcodes) as rows and gene names in columns (but file has gene names as rows)
adata = sc.read_text('rawdata/GSE181276_genes.counts_for_GEO_uploading.txt').T

In [16]:
# adata.obs contains cell barcodes
adata.obs

""
WT_AAACCCAAGCATCTTG
WT_AAACCCAAGGGTTGCA
WT_AAACCCAGTCCGAAAG
WT_AAACCCAGTCCGTTTC
WT_AAACCCAGTGTGGTCC
...
D7_analysis_TTTGTTGGTGAGCAGT
D7_analysis_TTTGTTGGTGCCTGAC
D7_analysis_TTTGTTGGTTGAGAGC
D7_analysis_TTTGTTGTCCTTATAC


In [17]:
#adata.var contains Gene Names
adata.var

""
Xkr4
Gm1992
Gm37381
Rp1
Sox17
...
AC168977.1
AC149090.1
CAAA01118383.1
Vmn2r122


In [18]:
# Shows size of count matrix (n_cells, n_genes)
adata.shape

(33681, 31053)

In [19]:
Total_gene_number = adata.shape[1]

In [20]:
# Filter to remove genes occuring in less than 10 cells
sc.pp.filter_genes(adata, min_cells=10)

In [21]:
Gene_Number_after_filtering = adata.shape[1]

In [22]:
Loss_of_genes = round((Total_gene_number-Gene_Number_after_filtering)/Total_gene_number*100,2)
Loss_of_genes

43.56

In [23]:
# Subsetting the highly variable genes to train the ScVI model
sc.pp.highly_variable_genes(adata, n_top_genes = 2000, subset = True, flavor='seurat_v3')

In [24]:
# adata shape after subsetting
adata

AnnData object with n_obs × n_vars = 33681 × 2000
    var: 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'

In [ ]:
# Train the SCVI Model (required for doublet removal model SOLO)
scvi.model.SCVI.setup_anndata(adata)
vae = scvi.model.SCVI(adata)
vae.train()

/Users/thorsten/.pyenv/versions/3.10.6/envs/SingleCell/lib/python3.10/site-packages/scvi/train/_trainrunner.py:69: UserWarning: `accelerator` has been automatically set to `cpu` although 'mps' exists. If you wish to run on mps backend, use explicitly accelerator=='mps' in train function.In future releases it will become default for mps supported machines.
  accelerator, lightning_devices, device = parse_device_args(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/thorsten/.pyenv/versions/3.10.6/envs/SingleCell/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/thorsten/.pyenv/versions/3.10.6/envs/SingleCell/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 

Training:   0%|          | 0/238 [00:00<?, ?it/s]

In [31]:
# Train the doublet class model (it has an stop inside if loss is not changing anymore)
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()

INFO     Creating doublets, preparing SOLO model.                                                                  


/Users/thorsten/.pyenv/versions/3.10.6/envs/SingleCell/lib/python3.10/site-packages/scvi/train/_trainrunner.py:69: UserWarning: `accelerator` has been automatically set to `cpu` although 'mps' exists. If you wish to run on mps backend, use explicitly accelerator=='mps' in train function.In future releases it will become default for mps supported machines.
  accelerator, lightning_devices, device = parse_device_args(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/thorsten/.pyenv/versions/3.10.6/envs/SingleCell/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/thorsten/.pyenv/versions/3.10.6/envs/SingleCell/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 

Training:   0%|          | 0/400 [00:00<?, ?it/s]

Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.209. Signaling Trainer to stop.


In [32]:
# Use predict to annotate the cell barcodes with the trained identifier and annotate it as a string in "doublet" or "singlet"
df = solo.predict()
df['prediction'] = solo.predict(soft=False)
df

/Users/thorsten/.pyenv/versions/3.10.6/envs/SingleCell/lib/python3.10/site-packages/torch/utils/_contextlib.py:116: UserWarning: Prior to scvi-tools 1.1.3, `SOLO.predict` with `soft=True` (the default option) returned logits instead of probabilities. This behavior has since been corrected to return probabiltiies. The previous behavior can be replicated by passing in `return_logits=True`.
  return func(*args, **kwargs)
/Users/thorsten/.pyenv/versions/3.10.6/envs/SingleCell/lib/python3.10/site-packages/torch/utils/_contextlib.py:116: UserWarning: Prior to scvi-tools 1.1.3, `SOLO.predict` with `soft=True` (the default option) returned logits instead of probabilities. This behavior has since been corrected to return probabiltiies. The previous behavior can be replicated by passing in `return_logits=True`.
  return func(*args, **kwargs)


,doublet,singlet,prediction
WT_AAACCCAAGCATCTTG,0.006114,0.993886,singlet
WT_AAACCCAAGGGTTGCA,0.120150,0.879850,singlet
WT_AAACCCAGTCCGAAAG,0.188696,0.811304,singlet
WT_AAACCCAGTCCGTTTC,0.097345,0.902655,singlet
WT_AAACCCAGTGTGGTCC,0.003006,0.996994,singlet
...,...,...,...
D7_analysis_TTTGTTGGTGAGCAGT,0.027798,0.972202,singlet
D7_analysis_TTTGTTGGTGCCTGAC,0.266351,0.733649,singlet
D7_analysis_TTTGTTGGTTGAGAGC,0.101135,0.898865,singlet
D7_analysis_TTTGTTGTCCTTATAC,0.057880,0.942120,singlet


In [52]:
# Count how many cells are predicted as singlet or doublet
counts = df.groupby('prediction').count()

# Total number of cells
total = counts.sum().values[0]  # or len(df)

# Number of predicted doublets
num_doublets = df.groupby('prediction').count().loc['doublet','doublet']

# Percentage of doublets
percent_doublets = (num_doublets / total) * 100

print(f"Doublets: {num_doublets} / {total} ({percent_doublets:.2f}%)")

Doublets: 3811 / 33681 (11.31%)


In [36]:
#Calculating the difference of doublet and singlet score to identify cells that have high scores in both (close to 0)
df['dif'] = df.doublet - df.singlet
df

,doublet,singlet,prediction,dif
WT_AAACCCAAGCATCTTG,0.006114,0.993886,singlet,-0.987771
WT_AAACCCAAGGGTTGCA,0.120150,0.879850,singlet,-0.759700
WT_AAACCCAGTCCGAAAG,0.188696,0.811304,singlet,-0.622608
WT_AAACCCAGTCCGTTTC,0.097345,0.902655,singlet,-0.805311
WT_AAACCCAGTGTGGTCC,0.003006,0.996994,singlet,-0.993988
...,...,...,...,...
D7_analysis_TTTGTTGGTGAGCAGT,0.027798,0.972202,singlet,-0.944404
D7_analysis_TTTGTTGGTGCCTGAC,0.266351,0.733649,singlet,-0.467298
D7_analysis_TTTGTTGGTTGAGAGC,0.101135,0.898865,singlet,-0.797729
D7_analysis_TTTGTTGTCCTTATAC,0.057880,0.942120,singlet,-0.884240


In [ ]:
# Count how many cells are predicted as singlet or doublet
counts = df.groupby('prediction').count()

# Total number of cells
total = counts.sum().values[0]  # or len(df)

# Number of predicted doublets
num_doublets = counts.loc['doublet'].values[0]

# Percentage of doublets
percent_doublets = (num_doublets / total) * 100

print(f"Doublets: {num_doublets} / {total} ({percent_doublets:.2f}%)")